This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [317]:
import os
from langchain_openai import OpenAI, OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableSequence
import numpy as np
import re
import json

In [318]:
os.environ["OPENAI_API_KEY"] = "API-KEY"

In [319]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    max_tokens=2000
)
embeddings = OpenAIEmbeddings()

In [330]:
def generate_listings(num_listings=10):
    prompt = PromptTemplate(
            input_variables=["n"],
            template="""Generate {n} diverse and concise real estate listings. Each listing should be a JSON object with the following structure:
            {{
                "location": "City, Neighborhood",
                "price": "$X,XXX,XXX",
                "bedrooms": X,
                "bathrooms": X,
                "size": "X,XXX sqft",
                "year_built": XXXX,
                "type": "house/apartment/etc",
                "features": ["feature1", "feature2", "feature3"],
                "condition": "new/renovated/etc",
                "description": "An informative description focusing on key selling points, written in 2 paragraphs. The first paragraph should highlight the main features and the uniqueness of the property. The second paragraph should provide additional details such as nearby amenities, transportation options, and potential uses of the space."
            }}
            Ensure variety in all aspects, including prices, locations, and features.
            Return the listings as a JSON array.
            """
    )
    chain = RunnableSequence(prompt, llm, StrOutputParser())
    listings_json = chain.invoke({"n": num_listings})
    listings = json.loads(listings_json)
    return listings

In [321]:
def normalize_l2(x):
    """Normalize vector to unit length."""
    return x / np.linalg.norm(x, ord=2, axis=-1, keepdims=True)

def create_vector_db(listings):
    texts = [json.dumps(listing) for listing in listings]
    
    # Get embeddings
    embedded_texts = embeddings.embed_documents(texts)
    
    # Normalize embeddings
    normalized_embeddings = [normalize_l2(np.array(emb)) for emb in embedded_texts]
    
    # Create FAISS index
    index = FAISS.from_embeddings(
        text_embeddings=list(zip(texts, normalized_embeddings)),
        embedding=embeddings,
        metadatas=listings
    )
    
    return index

In [322]:
def semantic_search(db, preferences, k=3):
    # Normalize the query vector
    query_vector = normalize_l2(np.array(embeddings.embed_query(preferences)))
    
    # Perform the search
    results = db.similarity_search_with_score_by_vector(query_vector, k=k)
    
    return [doc.metadata for doc, score in results]

In [323]:
def generate_personalized_description(listing, preferences):
    prompt = PromptTemplate(
        input_variables=["listing", "preferences"],
        template="Given the following property description:\n{listing}\n\nAnd the buyer's preferences:\n{preferences}\n\nGenerate a personalized description that highlights the aspects of the property that match the buyer's preferences, without changing the factual information."
    )
    chain = RunnableSequence(prompt, llm, StrOutputParser())
    return chain.invoke({"listing": listing, "preferences": preferences})


def generate_personalized_description(listing, preferences):
    prompt = PromptTemplate(
        input_variables=["listing", "preferences"],
        template="""Given the following detailed property description:
    {listing}

    And the buyer's preferences:
    {preferences}

    Generate a personalized description that highlights the aspects of the property matching the buyer's preferences. The description should:

    1. Maintain all factual information accurately.
    2. Emphasize features that match the buyer's preferences.
    3. Briefly mention any additional features that might be of interest.
    4. Have a persuasive yet honest tone.
    5. Be 3-4 paragraphs in length.

    Do not add or modify any property features."""
    )
    chain = RunnableSequence(prompt, llm, StrOutputParser())
    return chain.invoke({"listing": json.dumps(listing), "preferences": preferences})

In [327]:
def main():
    # Generate listings
    print("Generating real estate listings...")
    listings = generate_listings()
    
    # Create vector database
    print("Creating vector database...")
    db = create_vector_db(listings)
    
    # Simulate buyer preferences
    buyer_preferences = "I'm looking for a house near the city center, preferably renovated, with a garden."
    # Perform semantic search
    print("Performing semantic search...")
    matching_listings = semantic_search(db, buyer_preferences,k = 3)
    # Generate personalized descriptions
    print("Generating personalized descriptions...")
    for i, listing in enumerate(matching_listings, 1):
        personalized_description = generate_personalized_description(listing, buyer_preferences)
        print(f"\nListing {i}:")
        print("Original details:")
        for key, value in listing.items():
            print(f"  {key}: {value}")
        print("\nPersonalized description:")
        print(personalized_description)
        print("-" * 50)

In [331]:
main()

Generating real estate listings...
Creating vector database...
Performing semantic search...
Generating personalized descriptions...

Listing 1:
Original details:
  location: Austin, Zilker
  price: $600,000
  bedrooms: 2
  bathrooms: 2
  size: 1,200 sqft
  year_built: 2010
  type: duplex
  features: ['hardwood floors', 'private yard', 'modern kitchen']
  condition: renovated
  description: Live in the heart of Austin in this renovated Zilker duplex. With its hardwood floors, private yard, and modern kitchen, this property offers the perfect blend of comfort and style. Just a short walk to Barton Springs and downtown, this is urban living at its best.

Personalized description:
Welcome to your dream home in the heart of Austin's Zilker neighborhood! This beautifully renovated duplex is located just a short walk from Barton Springs and downtown, offering the perfect blend of urban living and tranquility. 

As you step inside, you'll be greeted by the gleaming hardwood floors that flow t